In [ ]:
import pandas as pd

In [ ]:
# CheXpert images can be found: https://stanfordaimi.azurewebsites.net/datasets/8cbd9ed4-2eb9-4565-affc-111cf4f7ebe2
data_df = pd.read_csv('chexpert_train.csv')

# Demographic labels can be found: https://stanfordaimi.azurewebsites.net/datasets/192ada7c-4d43-466e-b8bb-b81992bb80cf
demo_df = pd.DataFrame(pd.read_excel("CHEXPERT DEMO.xlsx", engine='openpyxl')) #pip install openpyxl
